<a href="https://colab.research.google.com/github/Devansh-Vaidya/FeatureMapping/blob/main/COMP478.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Image Processing - Feature Mapping</h1>


Import Python Modules:

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
from skimage.transform import resize
from skimage.util import img_as_float
from skimage.color import rgb2gray
from skimage.feature import match_descriptors, SIFT
import cv2

Utility Functions:

In [2]:
def visualize_box(template, target, H):
    # visualize the detected box of the template in target
    nrow, ncol = template.shape[:2]
    row = np.array([0, 0, nrow - 1, nrow - 1, 0]).astype(float)
    col = np.array([0, ncol-1, ncol-1, 0, 0]).astype(float)
    x = H[0, 0]*col + H[0, 1]*row + H[0, 2]
    y = H[1, 0]*col + H[1, 1]*row + H[1, 2]
    w = H[2, 0]*col + H[2, 1]*row + H[2, 2]
    x = x / w
    y = y / w
    x = np.round(x).astype(int)
    y = np.round(y).astype(int)
    plt.imshow(target)
    plt.plot(x, y, 'r-')
    plt.show()


def visualize_match(template, target, locs1, locs2, matches):
    # assume template has fewer rows. Both template/target are grayscale
    assert(template.ndim == 2)
    assert(target.ndim == 2)
    nrow1, ncol1 = template.shape[:2]
    nrow2 = target.shape[0]
    template = np.pad(template, ((0, nrow2 - nrow1), (0, 0)))

    img = np.hstack((template, target))

    plt.imshow(img, cmap='gray')
    i1 = matches[:, 0]
    i2 = matches[:, 1]
    x1 = locs1[i1, 1]
    y1 = locs1[i1, 0]
    x2 = locs2[i2, 1] + ncol1
    y2 = locs2[i2, 0]
    plt.plot([x1, x2], [y1, y2])
    plt.show()

**Image Matching:**

In [3]:
def matchPics(I1, I2):
    sift_desc = SIFT()

    sift_desc.detect_and_extract(I1)
    locs1 = sift_desc.keypoints
    descs1 = sift_desc.descriptors

    sift_desc.detect_and_extract(rgb2gray(I2))
    locs2 = sift_desc.keypoints
    descs2 = sift_desc.descriptors

    matches = match_descriptors(descs1, descs2, max_ratio=0.6,
                                cross_check=True)

    return matches, locs1, locs2

**Compute Homography:**

In [ ]:
def computeH_ransac(matches, locs1, locs2):

    # Compute the best fitting homography using RANSAC given a list of matching pairs
    inliers = np.zeros((0, 2))
    bestH = None
    THRESHOLD = 0.75
    for i in range(500):
        # Selecting 4 random points
        pts = matches[np.random.choice(matches.shape[0], 4, replace=False)]

        # Computing homography using the 4 points from matches
        A = np.zeros((pts.shape[0] * 2, 9))
        for i in range(pts.shape[0]):
            x1 = locs1[pts[i, 0], 1]
            y1 = locs1[pts[i, 0], 0]
            x2 = locs2[pts[i, 1], 1]
            y2 = locs2[pts[i, 1], 0]
            A[i * 2, :] = [-x1, -y1, -1, 0, 0, 0, x2 * x1, x2 * y1, x2]
            A[i * 2 + 1, :] = [0, 0, 0, -x1, -y1, -1, y2 * x1, y2 * y1, y2]
        U, S, Vt = np.linalg.svd(A)
        H = np.reshape(Vt[8], (3, 3))
        N = []

        # Checking for inliers
        for i in range(len(matches)):
            p1 = np.transpose(
                np.matrix([locs1[matches[i, 0], 1], locs1[matches[i, 0], 0], 1]))
            p2 = np.transpose(
                np.matrix([locs2[matches[i, 1], 1], locs2[matches[i, 1], 0], 1]))

            estimatep2 = np.dot(H, p1)
            estimatep2 = estimatep2/estimatep2[2]
            error = p2 - estimatep2
            d = np.linalg.norm(error)

            if d < 0.75:
                N.append(i)

        if len(N) > len(inliers):
            inliers = N
            bestH = H

        if len(inliers) > (len(matches) * THRESHOLD):
            break

    return bestH, inliers

**Image Warping:**

In [ ]:
def compositeH(H, template, img):

    # Create a compositie image after warping the template image on top
    # of the image using homography
    height, width = img.shape[:2]

    # Create mask of same size as template
    mask = np.full_like(template, fill_value=255)

    # Warp mask by appropriate homography
    warped_mask = cv2.warpPerspective(mask, H, (width, height))
    inv_mask = cv2.bitwise_not(warped_mask)

    # Warp template by appropriate homography
    warped_template = cv2.warpPerspective(
        template, H, (width, height))

    # Use mask to combine the warped template and the image
    combine_mask = cv2.bitwise_and(img, inv_mask)
    composite_img = cv2.add(combine_mask, warped_template)

    return composite_img

**Reading Images:**

In [ ]:
cv_desk = img_as_float(io.imread('/content/book1_desk_2.jpg'))
cv_desk_gray = rgb2gray(cv_desk)
as_float = img_as_float(io.imread('/content/book1_cover.jpg'))
cv_cover = rgb2gray(as_float)
hp_cover = img_as_float(io.imread('/content/drive/MyDrive/Colab Notebooks/images/hp_cover.jpg'))

FileNotFoundError: ignored

**Finding matches between 2 images:**

In [ ]:
# finding candidate matching pairs between two images
matches, locs1, locs2 = matchPics(cv_cover, cv_desk)

NameError: ignored

**Ouput of image matching:**

In [ ]:
# visualize raw matching result
visualize_match(cv_cover, cv_desk_gray, locs1, locs2, matches)

NameError: ignored

**Resizing the other book cover:**

In [ ]:
# resize hp_cover to have the same size of cv_cover
hp_cover_resize = resize(hp_cover, cv_cover.shape, anti_aliasing=True)

NameError: ignored

**Applying RANSAC and Estimating Homography Matrix:**

In [ ]:
# use RANSAC to estimate homograhy and find inlier matches
bestH, inliers = computeH_ransac(matches, locs1, locs2)

NameError: ignored

**Checking result of RANSAC and boundry of target image:**

In [ ]:
# visualize matching result after RANSAC
visualize_match(cv_cover, cv_desk_gray, locs1, locs2, matches[inliers, :])

# visualize the bounding box in target image
visualize_box(cv_cover, cv_desk, bestH)

NameError: ignored

**Warping Image:**

In [ ]:
# create final composite image
composite_img = compositeH(bestH, hp_cover_resize, cv_desk)

NameError: ignored

**Show Final Result:**

In [ ]:
# show the final composite image
plt.imshow(composite_img)
plt.show()

NameError: ignored